In [1]:
import os

import tensorflow as tf
import numpy as np

from tensorflow import keras
from tensorflow.keras import layers
from deeplab_mdl_def import DynamicUpsample
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-04-23 11:29:46.269521: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-23 11:29:46.269548: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-23 11:29:46.269575: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-23 11:29:46.276190: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [22]:
def DeeplabV3Plus_nicco(num_classes,
                        filters_conv1=24, filters_conv2=24,
                        filters_spp=128, filters_final=128,
                        dilated_conv_rates =[1, 4, 8, 16],
                        trainable_resnet=False,
                        path_model_trained: str = None,
                        transfer_learning: str = "last"):
    
    assert transfer_learning in ["last", "all"], "NICCO: Transfer learning policy not supported"
    model_input = keras.Input(shape=(None, None, 3))
    preprocessed = keras.applications.resnet50.preprocess_input(model_input)
    resnet50 = keras.applications.ResNet50(weights="imagenet", include_top=False, input_tensor=preprocessed)
    resnet50.trainable=trainable_resnet

    x = resnet50.get_layer("conv4_block6_2_relu").output
    input_b = resnet50.get_layer("conv2_block3_2_relu").output

    x1 = layers.GlobalAveragePooling2D()(x)
    x1 = layers.Reshape((1, 1, x.shape[-1]))(x1)
    x1 = layers.Conv2D(filters=filters_conv1, kernel_size=1, padding="same")(x1) #Perché qui c'è una conv2d?
    x1 = layers.BatchNormalization()(x1)
    x1 = DynamicUpsample()(x1, x)

    # Multiple dilated convolutions
    pyramids = []
    for rate in dilated_conv_rates:
        if rate == 1:
            pyramid = layers.Conv2D(filters=filters_spp, kernel_size=3, dilation_rate=rate, padding="same")(x)
            pyramid = layers.BatchNormalization()(pyramid)
            pyramids.append(pyramid)
        else:
            pyramid = layers.Conv2D(filters=filters_spp, kernel_size=3 + int(rate*(1/3)), dilation_rate=rate, padding="same")(x)
            pyramid = layers.BatchNormalization()(pyramid)
            pyramids.append(pyramid)

    x = layers.Concatenate(axis=-1)([x1] + pyramids)
    x = layers.Conv2D(filters=filters_spp, kernel_size=1, padding="same")(x)
    x = layers.BatchNormalization()(x)

    input_b = layers.Conv2D(filters=filters_conv2, kernel_size=1, padding="same")(input_b)
    input_b = layers.BatchNormalization()(input_b)

    input_a = DynamicUpsample()(x, input_b)

    x = layers.Concatenate(axis=-1)([input_a, input_b])
    x = layers.Conv2D(filters=filters_final, kernel_size=3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(filters=filters_final, kernel_size=3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = DynamicUpsample()(x, model_input)
    
    #resnet_output = resnet50.get_layer("conv4_block6_2_relu").output
    #resnet_output = layers.GlobalAveragePooling2D()(resnet_output)
    x = layers.GlobalAveragePooling2D()(x)
    #x = layers.Concatenate(axis=-1)([x, resnet_output])
    x = layers.Dense(50,activation="selu")(x)
    x = layers.Dense(25,activation="selu")(x)
    model_output = layers.Dense(num_classes,activation="sigmoid")(x)
    model = keras.Model(inputs=model_input, outputs=model_output)
    
    if path_model_trained is not None:
        model_from = tf.keras.models.load_model(path_model_trained, 
                                                custom_objects={'DynamicUpsample': DynamicUpsample})
        for l, l_old in zip(model.layers[:-3], model_from.layers):
            l.set_weights(l_old.get_weights())
            if transfer_learning=="last": l.trainable = False
    return model


In [23]:
model_name : str = "deeplabv3plus_face_segmentation_augmentation_class_weights_latest_fixConv_different_rate.h5"

In [24]:
model = tf.keras.models.load_model(model_name, custom_objects={'DynamicUpsample': DynamicUpsample})

In [25]:
model_nicco = DeeplabV3Plus_nicco(1)

In [10]:
for i, j in zip(model_nicco.get_weights(), model.get_weights()): 
    print(i.shape, "|", j.shape,  "|", np.array_equal(i.shape, j.shape), "|", np.array_equal(i,j))

(7, 7, 3, 64) | (7, 7, 3, 64) | True | False
(64,) | (64,) | True | False
(64,) | (64,) | True | False
(64,) | (64,) | True | False
(64,) | (64,) | True | False
(64,) | (64,) | True | False
(1, 1, 64, 64) | (1, 1, 64, 64) | True | False
(64,) | (64,) | True | False
(64,) | (64,) | True | False
(64,) | (64,) | True | False
(64,) | (64,) | True | False
(64,) | (64,) | True | False
(3, 3, 64, 64) | (3, 3, 64, 64) | True | False
(64,) | (64,) | True | False
(64,) | (64,) | True | False
(64,) | (64,) | True | False
(64,) | (64,) | True | False
(64,) | (64,) | True | False
(1, 1, 64, 256) | (1, 1, 64, 256) | True | False
(256,) | (256,) | True | False
(1, 1, 64, 256) | (1, 1, 64, 256) | True | False
(256,) | (256,) | True | False
(256,) | (256,) | True | False
(256,) | (256,) | True | False
(256,) | (256,) | True | False
(256,) | (256,) | True | False
(256,) | (256,) | True | False
(256,) | (256,) | True | False
(256,) | (256,) | True | False
(256,) | (256,) | True | False
(1, 1, 256, 64) | 

In [11]:
for l in model_nicco.layers:
    print(l.name, l.trainable, [i.shape for i in l.get_weights()])

input_2 False []
tf.__operators__.getitem_2 False []
tf.nn.bias_add_2 False []
conv1_pad False []
conv1_conv False [(7, 7, 3, 64), (64,)]
conv1_bn False [(64,), (64,), (64,), (64,)]
conv1_relu False []
pool1_pad False []
pool1_pool False []
conv2_block1_1_conv False [(1, 1, 64, 64), (64,)]
conv2_block1_1_bn False [(64,), (64,), (64,), (64,)]
conv2_block1_1_relu False []
conv2_block1_2_conv False [(3, 3, 64, 64), (64,)]
conv2_block1_2_bn False [(64,), (64,), (64,), (64,)]
conv2_block1_2_relu False []
conv2_block1_0_conv False [(1, 1, 64, 256), (256,)]
conv2_block1_3_conv False [(1, 1, 64, 256), (256,)]
conv2_block1_0_bn False [(256,), (256,), (256,), (256,)]
conv2_block1_3_bn False [(256,), (256,), (256,), (256,)]
conv2_block1_add False []
conv2_block1_out False []
conv2_block2_1_conv False [(1, 1, 256, 64), (64,)]
conv2_block2_1_bn False [(64,), (64,), (64,), (64,)]
conv2_block2_1_relu False []
conv2_block2_2_conv False [(3, 3, 64, 64), (64,)]
conv2_block2_2_bn False [(64,), (64,), (64

In [12]:
IMG_WIDTH=224
IMG_HEIGHT=224
BATCH_SIZE=128
EPOCHS=100
HIDDEN_UNITS=400
TRAINABLE_BASE = False
training_max_batches = None
use_pooling = False
use_validation_data_in_training = True
use_data_aug = False

real_train_path = "/homeRepo/tanfoni/Dataset_merged/Train/Real/"
generated_train_path = "/homeRepo/tanfoni/Dataset_merged/Train/Fake"
resize_dim = (IMG_WIDTH, IMG_HEIGHT)
model_input_shape = (IMG_WIDTH, IMG_HEIGHT, 3)

dataset_batch_size = BATCH_SIZE
train_batch_size = int(dataset_batch_size * 0.6)
validation_batch_size = int((dataset_batch_size - train_batch_size) * 0.6)
test_batch_size = dataset_batch_size - train_batch_size - validation_batch_size

parent_dir_train = "/homeRepo/tanfoni/Dataset_merged/Train/"
real_dir_train = "Real"
sint_dir_train = "Fake"

parent_dir_val = "/homeRepo/tanfoni/Dataset_merged/Valid/"
real_dir_val = "Real"
sint_dir_val = "Fake"

parent_dir_test = "/homeRepo/tanfoni/Dataset_merged/Test/"
real_dir_test = "Real"
sint_dir_test = "Fake"



print("Loading training images")
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    parent_dir_train,
    labels="inferred",
    class_names=[real_dir_train, sint_dir_train],
    label_mode='binary',
    color_mode='rgb',
    batch_size=train_batch_size,
    image_size=resize_dim,
    shuffle=True,
    seed=123,
)

if use_data_aug:
    print("Performing data augmentation on training images")
    data_augmentation = ImageDataGenerator(
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        label_mode='binary',
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    train_dataset_aug = data_augmentation.flow_from_directory(
        parent_dir_train,
        target_size=resize_dim,
        batch_size=train_batch_size,
        label_mode='binary',
        class_mode='binary',
        shuffle=True,
        seed=123
    )


print("Loading validation images")
val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    parent_dir_val,
    labels="inferred",
    class_names=[real_dir_val, sint_dir_val],
    label_mode='binary',
    color_mode='rgb',
    batch_size=validation_batch_size,
    image_size=resize_dim,
    shuffle=True,
    seed=123
)

print("Loading test images")
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    parent_dir_test,
    labels="inferred",
    class_names=[real_dir_test, sint_dir_test],
    label_mode='binary',
    color_mode='rgb',
    batch_size=test_batch_size,
    image_size=resize_dim,
    shuffle=True,
    seed=123
)

# Balancing dataset
n_fakes = len(os.listdir(os.path.join(parent_dir_train, sint_dir_train)))
n_reals = len(os.listdir(os.path.join(parent_dir_train, real_dir_train)))
total = n_fakes + n_reals
weight_for_0 = (1 / n_reals) * total / 2.0
weight_for_1 = (1 / n_fakes) * total / 2.0
class_weights = {0: weight_for_0, 1: weight_for_1}
print("Class weights: {}".format(class_weights))


Loading training images
Found 210000 files belonging to 2 classes.


2024-04-23 11:32:07.638708: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-04-23 11:32:07.641917: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 11.0.194, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2024-04-23 11:32:07.641943: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 11.0.194, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2024-04-23 11:32:07.641961: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 11.0.194, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2024-04-23 11:32:07.641983: E tens

Loading validation images
Found 30000 files belonging to 2 classes.
Loading test images
Found 60000 files belonging to 2 classes.
Class weights: {0: 1.5, 1: 0.75}


In [26]:
model_nicco.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 tf.__operators__.getitem_4  (None, None, None, 3)        0         ['input_4[0][0]']             
  (SlicingOpLambda)                                                                               
                                                                                                  
 tf.nn.bias_add_4 (TFOpLamb  (None, None, None, 3)        0         ['tf.__operators__.getitem_4[0
 da)                                                                ][0]']                        
                                                                                            

In [27]:
model_nicco.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
                    loss=keras.losses.BinaryCrossentropy(),
                    metrics=["accuracy",
                    keras.metrics.Precision(name='precision'),
                    keras.metrics.Recall(name='recall')])
model_nicco.fit(train_dataset, epochs=2, class_weight=class_weights, validation_data=val_dataset)

Epoch 1/2


2024-04-23 11:52:44.150708: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 11.0.194, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2024-04-23 11:52:44.152487: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-04-23 11:52:44.347621: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 11.0.194, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2024-04-23 11:52:44.510178: E tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:114] *** WARNING *** You are using ptxas 11.0.194, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or in

2764/2764 [==============================] - 300s 106ms/step - loss: 0.4753 - accuracy: 0.7205 - precision: 0.8971 - recall: 0.6560 - val_loss: 0.4860 - val_accuracy: 0.7637 - val_precision: 0.7850 - val_recall: 0.8890
Epoch 2/2
2764/2764 [==============================] - 293s 106ms/step - loss: 0.4409 - accuracy: 0.7506 - precision: 0.8978 - recall: 0.7063 - val_loss: 0.4087 - val_accuracy: 0.7937 - val_precision: 0.8292 - val_recall: 0.8697
